In [1]:
import numpy as np
import tensorflow as tf
from collections import deque
from matplotlib import pyplot as plt
import gym
env = gym.make('CartPole-v0')

# Constants defining our neural network
learning_rate = 1e-1
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

X = tf.placeholder(tf.float32, [None, input_size], name="input_x")

# First layer of weights
W1 = tf.get_variable("W1", shape=[input_size, output_size],
                     initializer=tf.contrib.layers.xavier_initializer())

# W2 = tf.get_variable("W2", shape=[input_size, input_size],
#                      initializer=tf.contrib.layers.xavier_initializer())

# W3 = tf.get_variable("W3", shape=[input_size, input_size],
#                      initializer=tf.contrib.layers.xavier_initializer())

# W4 = tf.get_variable("W4", shape=[input_size, input_size],
#                      initializer=tf.contrib.layers.xavier_initializer())

# W5 = tf.get_variable("W5", shape=[input_size, input_size],
#                      initializer=tf.contrib.layers.xavier_initializer())

# W6 = tf.get_variable("W6", shape=[input_size, output_size],
#                      initializer=tf.contrib.layers.xavier_initializer())

Qpred = tf.matmul(X, W1)
# wx1w2 = tf.matmul(xw1, W2)
# wx1w2w3 = tf.matmul(wx1w2, W3)
# wx1w2w3w4 = tf.matmul(wx1w2w3, W4)
# wx1w2w3w4w5 = tf.matmul(wx1w2w3w4, W5)  
            

# Qpred = tf.matmul(wx1w2w3w4w5, W6)

# We need to define the parts of the network needed for learning a policy
Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)

# Loss function
loss = tf.reduce_sum(tf.square(Y - Qpred))
# Learning
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Values for q learning
max_episodes = 1000
dis = 0.9
step_history = []


# Setting up our environment
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for episode in range(max_episodes):
    e = 1. / ((episode / 10) + 1)
    step_count = 0
    state = env.reset()
    done = False

    # The Q-Network training
    while not done:
        step_count += 1
        x = np.reshape(state, [1, input_size])
        # Choose an action by greedily (with e chance of random action) from
        # the Q-network
        Q = sess.run(Qpred, feed_dict={X: x})
        if np.random.rand(1) < e:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q)

        # Get new state and reward from environment
        next_state, reward, done, _ = env.step(action)
        if done:
            Q[0, action] = -100
        else:
            x_next = np.reshape(next_state, [1, input_size])
            # Obtain the Q' values by feeding the new state through our network
            Q_next = sess.run(Qpred, feed_dict={X: x_next})
            Q[0, action] = reward + dis * np.max(Q_next)

        # Train our network using target and predicted Q values on each episode
        sess.run(train, feed_dict={X: x, Y: Q})
        state = next_state

    step_history.append(step_count)
    print("Episode: {}  steps: {}".format(episode, step_count))
    # If last 10's avg steps are 500, it's good enough
    if len(step_history) > 10 and np.mean(step_history[-10:]) > 500:
        break

# See our trained network in action
observation = env.reset()
reward_sum = 0
while True:
    env.render()

    x = np.reshape(observation, [1, input_size])
    Q = sess.run(Qpred, feed_dict={X: x})
    action = np.argmax(Q)

    observation, reward, done, _ = env.step(action)
    reward_sum += reward
    if done:
        print("Total score: {}".format(reward_sum))
        break

C:\Users\HW-steve\Anaconda3\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Episode: 0  steps: 12
Episode: 1  steps: 18
Episode: 2  steps: 19
Episode: 3  steps: 10
Episode: 4  steps: 11
Episode: 5  steps: 12
Episode: 6  steps: 13
Episode: 7  steps: 13
Episode: 8  steps: 10
Episode: 9  steps: 13
Episode: 10  steps: 12
Episode: 11  steps: 10
Episode: 12  steps: 9
Episode: 13  steps: 13
Episode: 14  steps: 12
Episode: 15  steps: 12
Episode: 16  steps: 9
Episode: 17  steps: 14
Episode: 18  steps: 14
Episode: 19  steps: 13
Episode: 20  steps: 27
Episode: 21  steps: 32
Episode: 22  steps: 29
Episode: 23  steps: 54
Episode: 24  steps: 25
Episode: 25  steps: 82
Episode: 26  steps: 72
Episode: 27  steps: 37
Episode: 28  steps: 114
Episode: 29  steps: 29
Episode: 30  steps: 65
Episode: 31  steps: 31
Episode: 32  steps: 50
Episode: 33  steps: 40
Episode: 34  steps: 63
Episode: 35  steps: 46
Episode: 36  steps: 42
Episode: 37  steps: 42
Episode: 38  steps: 38
Episode: 39  steps: 43
Episode: 40  steps: 52
Episode: 41  steps: 31
Episode: 42  steps: 43
Episode: 43  steps: 46

Episode: 350  steps: 26
Episode: 351  steps: 21
Episode: 352  steps: 24
Episode: 353  steps: 18
Episode: 354  steps: 36
Episode: 355  steps: 12
Episode: 356  steps: 19
Episode: 357  steps: 18
Episode: 358  steps: 59
Episode: 359  steps: 35
Episode: 360  steps: 84
Episode: 361  steps: 31
Episode: 362  steps: 26
Episode: 363  steps: 27
Episode: 364  steps: 42
Episode: 365  steps: 24
Episode: 366  steps: 36
Episode: 367  steps: 39
Episode: 368  steps: 30
Episode: 369  steps: 20
Episode: 370  steps: 34
Episode: 371  steps: 23
Episode: 372  steps: 24
Episode: 373  steps: 20
Episode: 374  steps: 26
Episode: 375  steps: 62
Episode: 376  steps: 33
Episode: 377  steps: 28
Episode: 378  steps: 49
Episode: 379  steps: 26
Episode: 380  steps: 32
Episode: 381  steps: 39
Episode: 382  steps: 25
Episode: 383  steps: 25
Episode: 384  steps: 21
Episode: 385  steps: 60
Episode: 386  steps: 28
Episode: 387  steps: 41
Episode: 388  steps: 24
Episode: 389  steps: 22
Episode: 390  steps: 23
Episode: 391  st

Episode: 692  steps: 40
Episode: 693  steps: 46
Episode: 694  steps: 75
Episode: 695  steps: 42
Episode: 696  steps: 48
Episode: 697  steps: 44
Episode: 698  steps: 29
Episode: 699  steps: 39
Episode: 700  steps: 65
Episode: 701  steps: 39
Episode: 702  steps: 32
Episode: 703  steps: 28
Episode: 704  steps: 88
Episode: 705  steps: 52
Episode: 706  steps: 48
Episode: 707  steps: 38
Episode: 708  steps: 47
Episode: 709  steps: 31
Episode: 710  steps: 42
Episode: 711  steps: 47
Episode: 712  steps: 47
Episode: 713  steps: 58
Episode: 714  steps: 50
Episode: 715  steps: 47
Episode: 716  steps: 68
Episode: 717  steps: 57
Episode: 718  steps: 67
Episode: 719  steps: 101
Episode: 720  steps: 37
Episode: 721  steps: 38
Episode: 722  steps: 53
Episode: 723  steps: 30
Episode: 724  steps: 52
Episode: 725  steps: 53
Episode: 726  steps: 32
Episode: 727  steps: 22
Episode: 728  steps: 45
Episode: 729  steps: 40
Episode: 730  steps: 9
Episode: 731  steps: 39
Episode: 732  steps: 10
Episode: 733  st

In [ ]:
plt.plot(step_history)